In [6]:
import json
import time

print('start')
with open("Pokemons-0ProfessorFig-23-06-2025.json") as f:
    data = json.load(f)
print('file open')
minified_data = {}
i = 0
last_time = time.time()
print('start moving')
for mon_id, mon in data.items():
    if(i%3000==0):
        nowish = time.time()
        print(i, nowish - last_time)
        last_time = nowish
    minified_data[mon_id] = {
        "mon_isshiny": mon.get("mon_isshiny"),
        "mon_cp": mon.get("mon_cp"),
        "mon_name": mon.get("mon_name"),
        "mon_number": mon.get("mon_number"),
        "mon_islucky": mon.get("mon_islucky")
    }

    if "mon_form" in mon:
        minified_data[mon_id]["mon_form"] = mon["mon_form"]
    if "mon_costume" in mon:
        minified_data[mon_id]["mon_costume"] = mon["mon_costume"]

    if "mon_alignment" in mon:
        minified_data[mon_id]["mon_alignment"] = mon["mon_alignment"]
    if "is_traded" in mon:
        minified_data[mon_id]["is_traded"] = mon["is_traded"]
    i = i + 1
print('done moving')
with open("Pokemons-0ProfessorFig-min.json", "w") as f:
    json.dump(minified_data, f, indent=2)
print('done dumping')

start
file open
start moving
0 0.00022125244140625
3000 0.004122734069824219
6000 0.0033371448516845703
done moving
done dumping


In [2]:
def split_large_json_object(input_path, out1_path, out2_path):
    import re

    def write_wrapper(file, is_first):
        file.write("{\n")
        return lambda line, is_last=False: file.write(
            ("" if is_first else ",") + line + ("\n" if is_last else "")
        )

    brace_depth = 0
    current_lines = []
    writing_first = True
    pair_count = 0
    halfway = None

    with open(input_path, "r") as infile:
        lines = infile.readlines()

    # Estimate halfway point from counting top-level keys
    key_lines = [line for line in lines if re.match(r'\s*".+?": \{', line)]
    halfway = len(key_lines) // 2

    out1 = open(out1_path, "w")
    out2 = open(out2_path, "w")
    write1 = write_wrapper(out1, is_first=True)
    write2 = write_wrapper(out2, is_first=True)

    collecting = False
    for line in lines:
        if re.match(r'\s*".+?": \{', line):
            collecting = True
            current_lines = [line]
            brace_depth = 1
            continue

        if collecting:
            current_lines.append(line)
            brace_depth += line.count("{") - line.count("}")
            if brace_depth == 0:
                # Decide which file to write to
                joined = "".join(current_lines).rstrip(",\n")
                if pair_count < halfway:
                    write1(joined)
                else:
                    write2(joined)
                pair_count += 1
                collecting = False

    out1.write("\n}\n")
    out2.write("\n}\n")
    out1.close()
    out2.close()

split_large_json_object(
    "Pokemons-0ProfessorFig-23-06-2025 copy.json",
    "Pokemons-0ProfessorFig-part1.json",
    "Pokemons-0ProfessorFig-part2.json"
)

